In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader

In [4]:
from synthetic_utils import train_epoch, validate_epoch, train_encoder_classifier_epoch

In [5]:
if torch.cuda.is_available():
    device="cuda"
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device="cpu"
    torch.set_default_tensor_type('torch.FloatTensor')

In [6]:
DATA_PATH = 'generated_data.pkl'

In [7]:
with open(DATA_PATH, 'rb') as f:
    imgs, shapes, colors = pickle.load(f)

In [8]:
imgs.shape, shapes.shape, colors.shape

((50000, 96, 96, 3), (50000, 1), (50000, 1))

In [9]:
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(21 * 21 * 16, 1000)
        self.fc2 = nn.Linear(1000, 128)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [10]:
class ClassNet(nn.Module):
    
    def __init__(self, input_size=128):
        super(ClassNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
encoder = LeNet()
classifier = ClassNet()

In [12]:
criterion = nn.BCEWithLogitsLoss()
opt_cls = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
opt_enc = optim.Adam(encoder.parameters(), lr=0.001, betas=(0.9, 0.999))

In [13]:
rs = np.random.RandomState(1791387)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
imgs_train, imgs_test, shapes_train, shapes_test, colors_train, colors_test = train_test_split(
    imgs, shapes, colors, test_size=0.25, stratify=shapes, random_state=rs,
)

In [16]:
imgs_train.shape, imgs_test.shape

((37500, 96, 96, 3), (12500, 96, 96, 3))

In [17]:
colors_train.shape, colors_test.shape

((37500, 1), (12500, 1))

In [18]:
shapes_train.shape, shapes_test.shape

((37500, 1), (12500, 1))

In [19]:
losses, acc = train_encoder_classifier_epoch(encoder, classifier, imgs_train, shapes_train, opt_enc, opt_cls, criterion, device)

ValueError: operands could not be broadcast together with shapes (64,) (60,) 

In [ ]:
losses, acc